In [121]:
import pandas as pd
import datetime
import os
from dbHelpers import createEngine
import uuid
import numpy as np
import warnings
from pangres import upsert

warnings.simplefilter(action='ignore', category=pd.errors.DtypeWarning)
warnings.simplefilter(
    action='ignore', category=pd.errors.SettingWithCopyWarning)


In [43]:
engine = createEngine()

In [44]:
scratch_dir = os.path.join('..','scratch')

In [45]:
fips_state_df = pd.read_sql('select * from npd.fips_state', con = engine)
fips_state_df.set_index('abbreviation', inplace=True)

In [46]:
def show_or_load(df, table_name, schema_name, load=False):
    if load:
        print(f'Loading {table_name}')
        df.to_sql(table_name, schema = schema_name, con = engine, if_exists='append', index=False)
    else:
        df.head()

In [47]:
primary_to_bool = {1: True, 0: False}


def convertBool(val):
    if val in primary_to_bool.keys():
        return primary_to_bool[val]
    else:
        return False
    
def val_or_nan(df, index, column):
    if index in df.index:
        return df.loc[index][column]
    else:
        return np.nan

In [48]:
def handleDeactivatedNPIs(index, df):
    entity_type_code = df.loc[index]['entity_type_code']
    if not df.loc[index]['entity_type_code'].isnull():
        return df.loc[index]['entity_type_code']
    else:
        if df.loc[index]['Provider Last Name (Legal Name)'].isnull():
            return 2
        else:
            return 1
        df.loc[index]

In [58]:
current_date = datetime.datetime.now()
current_month = current_date.strftime("%B")
current_year = current_date.year
csv_version = f'{current_month}_{current_year}_V2'
nppes_dir = os.path.join(scratch_dir,'nppes')

# Download and unzip the NPPES CSV files
# zipData = requests.get(f'https://download.cms.gov/nppes/NPPES_Data_Dissemination_{csv_version}.zip').content
# with zipfile.ZipFile(io.BytesIO(zipData), 'r') as zip_file:
#    zip_file.extractall(nppes_dir)
main_files = [f for f in os.listdir(nppes_dir) if 'npidata_pfile' in f and '_fileheader' not in f]
main_files.sort()
latest_main_file = main_files[-1]

npi_df = pd.read_csv(os.path.join(nppes_dir, latest_main_file), usecols = ['Provider Last Name (Legal Name)', 'NPI', 'Entity Type Code', 'Replacement NPI', 'Provider Enumeration Date', 'Last Update Date',
                   'NPI Deactivation Reason Code', 'NPI Deactivation Date', 'NPI Reactivation Date', 'Certification Date'])
npi_df_renamed = npi_df.rename(columns={
        'NPI': 'npi',
        'Entity Type Code': 'entity_type_code',
        'Replacement NPI': 'replacement_npi',
        'Provider Enumeration Date': 'enumeration_date',
        'Last Update Date': 'last_update_date',
        'NPI Deactivation Reason Code': 'deactivation_reason_code',
        'NPI Deactivation Date': 'deactivation_date',
        'NPI Reactivation Date': 'reactivation_date',
        'Certification Date': 'certification_date'
    })
deactivated_npi1_condition = (npi_df_renamed['entity_type_code'].isnull())&~(npi_df_renamed['Provider Last Name (Legal Name)'].isnull())
deactivated_npi2_condition = (npi_df_renamed['entity_type_code'].isnull())&(npi_df_renamed['Provider Last Name (Legal Name)'].isnull())
npi_df_renamed.loc[deactivated_npi1_condition, ['entity_type_code', 'enumeration_date', 'last_update_date']] = [1, '1900-01-01', '1900-01-01']
npi_df_renamed.loc[deactivated_npi2_condition, ['entity_type_code', 'enumeration_date', 'last_update_date']] = [2, '1900-01-01', '1900-01-01']
del npi_df_renamed['Provider Last Name (Legal Name)']

In [50]:
df_dict={}
for f in os.listdir(os.path.join(scratch_dir, 'halloween_data')):
    if '.csv' in f:
        tablename = f.split('.csv')[0]
        df = pd.read_csv(os.path.join(scratch_dir,'halloween_data',f), na_values=[''], keep_default_na=False)
        df_dict[f]=df
        #df.to_sql(tablename, index=False, schema = 'raw_csv', con = engine, if_exists='replace')

In [51]:
practitioner_df = df_dict['practitioner.csv']
#note: we can do this because each practitioner only appears once in this table
practitioner_df['id'] = [uuid.uuid4() for i in practitioner_df.index]
practitioner_df_renamed = practitioner_df.rename(columns = {'gender_code': 'sex', 'name_prefix': 'prefix', 'name_suffix': 'suffix'})
practitioner_df_renamed['name_use_id'] = 1
practitioner_taxonomy_df = df_dict['practitionerrole.csv']
merged_taxonomy_df = practitioner_taxonomy_df.merge(practitioner_df_renamed, left_on = 'practitioner_id', right_on = 'npi', suffixes = ('tax', 'individual')) 
merged_taxonomy_df = merged_taxonomy_df.loc[merged_taxonomy_df['state_code']!='ZZ']
merged_taxonomy_df['state_code'] = merged_taxonomy_df['state_code'].apply(lambda x: val_or_nan(fips_state_df, x, 'id'))
merged_taxonomy_df_renamed = merged_taxonomy_df.rename(columns={'idindividual': 'individual_id', 'taxonomy_code':'nucc_code'})
provider_to_taxonomy_df = merged_taxonomy_df_renamed[['individual_id', 'nucc_code', 'is_primary']]
provider_to_taxonomy_df['is_primary'] = provider_to_taxonomy_df['is_primary'].apply(lambda x: convertBool(x))
dedup_taxonomy_df = provider_to_taxonomy_df.sort_values(by='is_primary', ascending=False)[
        ['individual_id', 'nucc_code', 'is_primary']].drop_duplicates(subset=['nucc_code', 'individual_id'])
dedup_taxonomy_df['id'] = [uuid.uuid4() for i in dedup_taxonomy_df.index]
credential_df = provider_to_taxonomy_df.merge(merged_taxonomy_df_renamed, on = ['individual_id', 'nucc_code'], suffixes = ('tax', 'cred'))
credential_df_renamed = credential_df.rename(columns={'idtax': 'provider_to_taxonomy_id'})

In [ ]:
organization_df = df_dict['organization.csv']
organization_df['is_primary'] = True
organization_df_renamed = organization_df.rename(columns={'id':'old_org_id', 'parent_id':'old_parent_id', 'organization_name':'name'})
organization_df_renamed.set_index(['old_org_id'], inplace=True)
organization_df_renamed['org_id'] = [uuid.uuid4() for i in organization_df_renamed.index]
organization_df_renamed['org_parent_id'] = organization_df_renamed['old_parent_id'].apply(lambda x: val_or_nan(organization_df_renamed, x, 'org_id'))
organization_npi_df = df_dict['organization_npi.csv']
organization_npi_df_renamed = organization_npi_df.rename(columns={'organization_id':'old_org_id'})
organization_npi_df_renamed['id'] = [uuid.uuid4() for i in organization_npi_df_renamed.index]
clinical_organization_df = organization_npi_df_renamed.merge(organization_df_renamed, on='old_org_id', how='outer')
clinical_organization_df_renamed = clinical_organization_df.rename(columns={'org_id':'parent_id'})
other_organization_df = organization_df_renamed.rename(columns = {'org_id':'id', 'org_parent_id': 'parent_id'})


In [87]:
endpoint_df = df_dict['endpoint.csv']
endpoint_df_renamed = endpoint_df.rename(columns={'id':'endpoint_id','fhir_url':'address'})
ehr_vendor_df = endpoint_df.drop_duplicates(subset='vendor_name')
ehr_vendor_df['id'] = [uuid.uuid4() for i in ehr_vendor_df.index]
ehr_vendor_df_renamed = ehr_vendor_df.rename(columns={'vendor_name':'name'})
ehr_vendor_df_renamed.set_index('name', inplace=True, drop=False)
endpoint_df_renamed['ehr_vendor_id'] = endpoint_df_renamed['vendor_name'].apply(lambda x: ehr_vendor_df_renamed.loc[x]['id'])
endpoint_df_renamed['environment_type_id'] = 'prod'
endpoint_df_renamed['endpoint_connection_type_id'] = 'hl7-fhir-rest'
endpoint_df_renamed['id'] = [uuid.uuid4() for i in endpoint_df_renamed.index]

In [88]:
org_to_endpoint_df = df_dict['organization_endpoint.csv']
merged_org_to_endpoint_df = org_to_endpoint_df.merge(endpoint_df_renamed, on = 'endpoint_id', how='outer').merge(clinical_organization_df_renamed, left_on = 'organization_npi', right_on = 'npi', suffixes = ('endpoint', 'organization'), how='outer')
merged_org_to_endpoint_df= merged_org_to_endpoint_df[['idendpoint', 'idorganization']].rename(columns = {'idendpoint': 'endpoint_instance_id', 'idorganization':'organization_id'})

In [ ]:
address_df = df_dict['location.csv']
address_df_renamed = address_df.rename(columns={'id':'address_us_id', 'line':'delivery_line_1', 'postalcode':'zipcode', 'city':'city_name'})
address_df_renamed['id']= [uuid.uuid4() for i in address_df_renamed.index]
address_df_renamed = address_df_renamed.loc[(address_df_renamed['state'] != 'FM') & (address_df_renamed['state'] != '~') & (address_df_renamed['state'] != 'UK') & (address_df['state'] != 'MH')]
address_df_renamed['state_code'] = address_df_renamed['state'].apply(lambda x: fips_state_df.loc[x]['id'])
location_npi_df = df_dict['npi_location.csv']
merged_df_1 = location_npi_df.merge(address_df_renamed, left_on='location_id', right_on = 'address_us_id', how='outer')
merged_df_2 = merged_df_1.merge(npi_df_renamed, on = 'npi', suffixes=('address','npi'), how='outer')
merged_df_3 = merged_df_2.merge(practitioner_df_renamed, on = 'npi', suffixes = ('address', 'individual'), how='outer')
merged_location_df = merged_df_3.merge(clinical_organization_df_renamed, on = 'npi', suffixes = ('address', 'organization'), how='outer')
merged_location_df_renamed = merged_location_df.rename(columns={'idaddress':'address_id', 'idindividual':'individual_id', 'id':'organization_id', 'nameaddress':'name'})
merged_location_df_renamed['address_use_id'] = 2
individual_to_address_df = merged_location_df_renamed[['address_id','individual_id', 'address_use_id']].dropna(how='any')
location_df = merged_location_df_renamed[['address_id','organization_id','name', 'address_use_id']].dropna(how='any')
location_df['id'] = [uuid.uuid4() for i in location_df.index]
location_to_endpoint_df = location_df.merge(merged_org_to_endpoint_df, on = 'organization_id', how='outer')[['id', 'endpoint_instance_id']].dropna(how = 'any').rename(columns = {'id':'location_id'})


In [90]:
provider_to_organization_df = df_dict['personal_npi_to_organizational_npi.csv']
merged_provider_to_org_df = provider_to_organization_df.merge(practitioner_df_renamed, left_on = 'personal_npi', right_on = 'npi', how='outer').merge(clinical_organization_df_renamed, left_on = 'organizational_npi', right_on = 'npi', suffixes = ('individual', 'organization'), how='outer')
provider_to_org_df_renamed = merged_provider_to_org_df.rename(columns = {'idindividual':'individual_id', 'idorganization':'organization_id'})
provider_to_org_df_renamed['id'] = [uuid.uuid4() for i in provider_to_org_df_renamed.index]
provider_to_org_df_renamed['relationship_type_id'] = 2
provider_to_location_df = provider_to_org_df_renamed.merge(location_df, on='organization_id', how='outer')
provider_to_location_df['id'] = [uuid.uuid4() for i in provider_to_location_df.index]

In [ ]:
schema_name = 'npd'
load = True

# load npi
#show_or_load(npi_df_renamed, 'npi', schema_name, load)

# load individual
#show_or_load(practitioner_df_renamed[['id', 'sex']], 'individual', schema_name, load)
practitioner_df_renamed_renamed = practitioner_df_renamed.rename(columns={'id':'individual_id'})

# load individual_to_name
#show_or_load(practitioner_df_renamed_renamed[['individual_id', 'first_name', 'middle_name', 'last_name', 'prefix', 'suffix', 'name_use_id']], 'individual_to_name', schema_name, load)

# load provider
#show_or_load(practitioner_df_renamed_renamed.merge(npi_df_renamed, on = 'npi', how='inner')[['npi', 'individual_id']], 'provider', schema_name, load)

# load organization
#show_or_load(other_organization_df[['id']], 'organization', schema_name, load)
other_organization_df.set_index('id', drop=False, inplace=True)
#######upsert(df = other_organization_df[['parent_id']], con = engine, schema = schema_name, if_row_exists='update', table_name = 'organization')
#show_or_load(clinical_organization_df_renamed[['id']], 'organization', schema_name, load)
clinical_organization_df_renamed.set_index('id', drop=False, inplace=True)
#######upsert(df = clinical_organization_df_renamed[['parent_id']], con = engine, schema = schema_name, if_row_exists='update', table_name = 'organization')

other_organization_df_renamed = other_organization_df.rename(columns={'id':'organization_id', 'organization_name':'name'})
clinical_organization_df_renamed_renamed = clinical_organization_df_renamed.rename(columns={'id':'organization_id'})

# load organization_to_name
#show_or_load(other_organization_df_renamed[['organization_id', 'name', 'is_primary']], 'organization_to_name', schema_name, load)
#show_or_load(clinical_organization_df_renamed_renamed[['organization_id', 'name', 'is_primary']], 'organization_to_name', schema_name, load)

# load clinical_organization
#show_or_load(clinical_organization_df_renamed_renamed[['organization_id', 'npi']], 'clinical_organization', schema_name, load)

# load ehr_vendor
#show_or_load(ehr_vendor_df_renamed[['id', 'name']], 'ehr_vendor', schema_name, load)

# load endpoint_instance
#show_or_load(endpoint_df_renamed[['id', 'ehr_vendor_id', 'address', 'endpoint_connection_type_id', 'environment_type_id']], 'endpoint_instance', schema_name, load)

# load address_us
#show_or_load(address_df_renamed[['address_us_id', 'delivery_line_1','city','state_code','zipcode']].rename(columns={'address_us_id':'id', 'city':'city_name'}), 'address_us', schema_name, load)

# load address
#show_or_load(address_df_renamed[['id', 'address_us_id']], 'address', schema_name, load)

# load individual_to_address
#show_or_load(individual_to_address_df, 'individual_to_address', schema_name, load)

# load organization_to_address
#show_or_load(location_df[['address_id','organization_id', 'address_use_id']], 'organization_to_address', schema_name, load)

# load location

#show_or_load(location_df[['id','address_id','organization_id']], 'location', schema_name, load)

# load location_to_endpoint
#show_or_load(location_to_endpoint_df, 'location_to_endpoint', schema_name, load)

# load provider_to_organization
show_or_load(provider_to_org_df_renamed.dropna(how='any'), 'provider_to_organization', schema_name, load)

# load provider_to_location
show_or_load(provider_to_location_df.dropna(how='any'), 'provider_to_location', schema_name, load)

# load provider_to_taxonomy
show_or_load(dedup_taxonomy_df, 'provider_to_taxonomy', schema_name, load)

# load provider_to_credential
show_or_load(credential_df_renamed[['license_number', 'state_code', 'provider_to_taxonomy_id']], 'provider_to_credential', schema_name, load)

Loading organization_to_address
Loading location
Loading location_to_endpoint
Loading provider_to_organization


ProgrammingError: (psycopg2.errors.UndefinedColumn) column "personal_npi" of relation "provider_to_organization" does not exist
LINE 1: INSERT INTO npd.provider_to_organization (personal_npi, orga...
                                                  ^

[SQL: INSERT INTO npd.provider_to_organization (personal_npi, organizational_npi, affiliation_source, individual_id, npiindividual, last_name, first_name, middle_name, prefix, suffix, credential_text, is_sole_proprietor, sex, name_use_id, old_org_id, npior ... 537379 characters truncated ... mary__999)s, %(parent_id__999)s, %(org_parent_id__999)s, %(id__999)s, %(relationship_type_id__999)s)]
[parameters: {'credential_text__0': 'D.O.', 'old_parent_id__0': 745046.0, 'is_primary__0': True, 'id__0': UUID('ad4e6686-2c56-4887-bdd4-0bd78b006059'), 'organization_id__0': UUID('cc3b0648-6886-4494-8ecd-a636acd9f45d'), 'suffix__0': 'JR.', 'individual_id__0': UUID('9287d4a1-69d3-4961-9fa7-9907effc94ee'), 'prefix__0': 'DR.', 'organizational_npi__0': 1

In [119]:
def findIDorParent(id, df):
    return df.loc[(df['id']==id)|(df['parent_id']==id)]

In [130]:
location_df

,address_id,organization_id,name,address_use_id,id
1,67eda5df-0dee-45ed-bc95-fc61cba29135,7c5b66ef-2e8a-4c66-9961-3cee4b166ed3,1700 Neuse Blvd,2,0695c2ad-d150-4ee6-bd47-e81cc312d78d
13,624c8fd2-903a-4c73-a5fe-e58a212d6e56,8effa053-9656-4163-b6c5-697a1064889d,411 Massachusetts Ave Ste 102,2,db6474c5-f8e7-49be-abed-b6a9a6d77f75
14,c9af7617-0825-4125-a286-cb6d746275ae,8effa053-9656-4163-b6c5-697a1064889d,411 MASS AVE,2,8275c20f-e159-4129-beb1-a834aef1eecd
15,b0a645f8-61b7-4fff-a623-3cb59f058f27,0262a30d-3cdf-4203-8ae4-9d97ed39c398,919 5th Ave Ste 100,2,4485ab1b-b4f4-46c4-b4ee-c301c0643d17
17,cb201207-787f-4fb9-bded-8afb0f0d7e3d,fbfce06a-f2a2-46f1-8053-273e2f319e14,3530 Camino del Rio N,2,d2704bcd-9704-4237-b648-bfa9d00481ba
...,...,...,...,...,...
11126497,1fc6810b-b256-4521-adca-de12224285f7,d05bc888-f7f7-462e-b68a-27c49c3ed66c,5300 E Main St Ste 201,2,d42b38a9-b1bc-4959-bd29-bfee4f68f67f
11126498,1dfa067a-2149-4389-9377-fd1435e5f039,d05bc888-f7f7-462e-b68a-27c49c3ed66c,5300 E MAIN ST,2,ac45738b-a3b0-438e-adeb-6d4847c8d6ee
11126503,87a4dab0-b9c1-4884-b00e-438be9dd2f6f,08c13dbe-3be7-40f9-b99e-e819aeafbd63,328 22ND AVE NORHT,2,f746eca3-c8d5-473d-997f-b509af57f0cc
11126504,029642c2-9b7c-4642-afdb-ac77e1ab9fa2,08c13dbe-3be7-40f9-b99e-e819aeafbd63,328 22nd Ave N,2,d0b0af15-c4a7-4635-9616-328b221f31a8


In [128]:
address_df_renamed.loc[address_df_renamed['id'].apply(lambda x: len(str(x))>10)]

,address_us_id,name,delivery_line_1,city,state,zipcode,address_source,id,state_code
0,763021339,12813 E 101st Pl N,12813 E 101st Pl N,Owasso,OK,74055,NPPES,6dc7974f-1b24-49d7-8739-78026f041ddb,40
1,-1388237163,250 Chateau Dr SW Ste 210,250 Chateau Dr SW Ste 210,Huntsville,AL,35801,NPPES,51bc2197-98ec-414f-9844-f0047bb8d46a,01
2,-2001517137,1657 Owen Dr,1657 Owen Dr,Fayetteville,NC,28304,NPPES,a8f6b64c-6ccb-47d7-bc28-73eab58b2687,37
3,-313208870,PO Box 13614,PO Box 13614,Reading,PA,19612,NPPES,3a474e7a-3fba-4c0b-9071-500d069c6ca0,42
4,705826484,6119 W Jefferson Blvd,6119 W Jefferson Blvd,Fort Wayne,IN,46804,NPPES,41a61f70-5fe3-4988-ac90-d3e81fd944ee,18
...,...,...,...,...,...,...,...,...,...
5227876,-636131155,13500 MIDWAY RD STE 350,13500 MIDWAY RD STE 350,DALLAS,TX,75244,PECOS_IDR,24533ee6-b5a5-4a9a-967f-6d6f0a377a8a,48
5227877,1546874111,"90 BERGEN STREET, DOCTORS OFFICE CENTER","90 BERGEN STREET, DOCTORS OFFICE CENTER",NEWARK,NJ,7101,PECOS_IDR,aba136d5-8176-40e2-bd30-e91f3d35310a,34
5227878,-1670264287,1711 W TEMPLE ST FL 1,1711 W TEMPLE ST FL 1,LOS ANGELES,CA,90026,PECOS_IDR,a23b2796-a7eb-4972-ac4c-fb3921ae80c9,06
5227879,1470423939,210 W MAIN ST,210 W MAIN ST,MT PLEASANT,UT,84647,PECOS_IDR,9c67d48d-e6aa-4c04-864f-55524e15fd75,49
